# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# pip install citipy

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from nithiya_api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "nithiya_output_data/cities.csv"

In [3]:
# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [4]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
# Change back to 1500 after basic testing
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

628

In [5]:
cities

['bluff',
 'naze',
 'nagato',
 'pangai',
 'san carlos de bariloche',
 'sidi ali',
 'tambovka',
 'ostrovnoy',
 'atuona',
 'lavrentiya',
 'mataura',
 'hermanus',
 'dikson',
 'saint anthony',
 'iracoubo',
 'schweinfurt',
 'tiksi',
 'ponta do sol',
 'kahului',
 'goderich',
 'astoria',
 'wattegama',
 'sentyabrskiy',
 'muromtsevo',
 'nome',
 'sal rei',
 'jamestown',
 'kaeo',
 'riyadh',
 'amderma',
 'mutsamudu',
 'harper',
 'sur',
 'iqaluit',
 'mount gambier',
 'punta arenas',
 'bejar',
 'arraial do cabo',
 'ewa beach',
 'provideniya',
 'airai',
 'ribeira grande',
 'kapaa',
 'ushuaia',
 'taolanaro',
 'port alfred',
 'mingguang',
 'ixtapa',
 'bairiki',
 'comodoro rivadavia',
 'ilulissat',
 'qasigiannguit',
 'bitung',
 'hilo',
 'albany',
 'rikitea',
 'butaritari',
 'akot',
 'dovolnoye',
 'busselton',
 'yellowknife',
 'tahlequah',
 'chicama',
 'kavieng',
 'codrington',
 'saskylakh',
 'grand gaube',
 'vaini',
 'vanimo',
 'olafsvik',
 'chumikan',
 'bambous virieux',
 'la crosse',
 'cape town',
 'r

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [6]:
# Save config information.
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"

# Build partial query URLx
query_url = f"{url}appid={weather_api_key}&units={units}&q="
print(query_url)

http://api.openweathermap.org/data/2.5/weather?appid=ab0436d4ad4aaccb31bc1903d6af96b9&units=imperial&q=


In [7]:
# set up lists to hold reponse info
lat = []
lon = []
temp = []
temp_max = []
humidity = []
cloudiness = []
wind_speed = []

# Loop through the list of cities and perform a request for data on each
for city in cities:
    response = requests.get(query_url + city).json()
    
    try:
        lat.append(response['coord']['lat'])
        lon.append(response['coord']['lon'])
        temp.append(response['main']['temp'])
        temp_max.append(response['main']['temp_max'])
        humidity.append(response['main']['humidity'])
        cloudiness.append(response['clouds']['all'])
        wind_speed.append(response['wind']['speed'])

    except:
        print(f"The latitude information received is: {lat}")
        print(f"The temperature information received is: {temp}")
        print(f"The max temperature information received is: {temp_max}")
        print(f"The longitude information received is: {lon}")
        print(f"The humidity information received is: {humidity}")
        print(f"The cloudiness information received is: {cloudiness}")
        print(f"The wind speed information received is: {wind_speed}")
        
        pass

The latitude information received is: [-46.6, 28.3667, 34.3833, -19.8, -41.1456, 36.0998, 50.0969, 68.0531, -9.8, 65.5833, -46.1927, -34.4187, 73.5069, 45.0205, 5.4802, 50.05, 71.6872, 32.6667, 20.8947, 43.7501, 46.1879, 6.7989]
The temperature information received is: [46.8, 81.52, 71.31, 76.93, 31.44, 73.78, 75.31, 45.46, 79.54, 37.31, 46.47, 65.8, 34.48, 60.96, 76.59, 79.83, 41.52, 72.54, 73.62, 46.74, 54.18, 90.52]
The max temperature information received is: [46.8, 81.52, 73.35, 76.93, 31.44, 73.78, 75.31, 45.46, 79.54, 37.31, 46.47, 65.8, 34.48, 65.05, 76.59, 81.84, 41.52, 74.01, 76.84, 51.91, 55.92, 90.52]
The longitude information received is: [168.3333, 129.4833, 131.2, -174.35, -71.3082, 0.4206, 128.0489, 39.5131, -139.0333, -171, 168.8643, 19.2345, 80.5464, -93.218, -53.2011, 10.2333, 128.8694, -17.1, -156.47, -81.7165, -123.8313, 81.4808]
The humidity information received is: [87, 91, 94, 75, 55, 67, 80, 68, 77, 94, 99, 70, 83, 60, 92, 60, 63, 74, 78, 81, 91, 34]
The cloudi

The latitude information received is: [-46.6, 28.3667, 34.3833, -19.8, -41.1456, 36.0998, 50.0969, 68.0531, -9.8, 65.5833, -46.1927, -34.4187, 73.5069, 45.0205, 5.4802, 50.05, 71.6872, 32.6667, 20.8947, 43.7501, 46.1879, 6.7989, 56.3724, 64.5011, 16.1833, 42.097, -35.1, 24.6877, 4.375, 22.5667, 63.7506, -37.8333, -53.15, 40.3864, -22.9661, 21.3156, 64.3833, -8.9266, 38.5167, 22.0752, -54.8, -33.5906, 32.7802, 20.7, 1.3292, -45.8667, 69.2167, 68.8193, 1.4451, 19.7297, 42.6001, -23.1203, 3.0707, 21.0964, 54.4958, -33.65, 62.456, 35.9154, -7.8447, -2.5744, -38.2667, 71.9167, -20.0064, -21.2, -2.6741, 64.8945, 54.7183, -20.3428, 43.8014, -33.9258, 36.1628, -25.2406, 49, 5.4112, 28.017, 24.6833, -42.7826, 69.4541, 71.2906, 55.7435, 70.9963, 13.2167, 37.1041, -16.4833, 70.6333, -0.6, 35.4275, 50.6789, -42.8794, 41.3712, 35.7333, -33.0153, 6.1333, 51.8]
The temperature information received is: [46.8, 81.52, 71.31, 76.93, 31.44, 73.78, 75.31, 45.46, 79.54, 37.31, 46.47, 65.8, 34.48, 60.96, 76.

The latitude information received is: [-46.6, 28.3667, 34.3833, -19.8, -41.1456, 36.0998, 50.0969, 68.0531, -9.8, 65.5833, -46.1927, -34.4187, 73.5069, 45.0205, 5.4802, 50.05, 71.6872, 32.6667, 20.8947, 43.7501, 46.1879, 6.7989, 56.3724, 64.5011, 16.1833, 42.097, -35.1, 24.6877, 4.375, 22.5667, 63.7506, -37.8333, -53.15, 40.3864, -22.9661, 21.3156, 64.3833, -8.9266, 38.5167, 22.0752, -54.8, -33.5906, 32.7802, 20.7, 1.3292, -45.8667, 69.2167, 68.8193, 1.4451, 19.7297, 42.6001, -23.1203, 3.0707, 21.0964, 54.4958, -33.65, 62.456, 35.9154, -7.8447, -2.5744, -38.2667, 71.9167, -20.0064, -21.2, -2.6741, 64.8945, 54.7183, -20.3428, 43.8014, -33.9258, 36.1628, -25.2406, 49, 5.4112, 28.017, 24.6833, -42.7826, 69.4541, 71.2906, 55.7435, 70.9963, 13.2167, 37.1041, -16.4833, 70.6333, -0.6, 35.4275, 50.6789, -42.8794, 41.3712, 35.7333, -33.0153, 6.1333, 51.8, 70.0167, 57.797, 52.4443, -21.6403, 64.75, -10.1667, 4.2333, -46.2817, -38.5473, 64.7507, 45.5234, -35.7167, 69.1304, 66.8333, -33.0393, -0.8

The latitude information received is: [-46.6, 28.3667, 34.3833, -19.8, -41.1456, 36.0998, 50.0969, 68.0531, -9.8, 65.5833, -46.1927, -34.4187, 73.5069, 45.0205, 5.4802, 50.05, 71.6872, 32.6667, 20.8947, 43.7501, 46.1879, 6.7989, 56.3724, 64.5011, 16.1833, 42.097, -35.1, 24.6877, 4.375, 22.5667, 63.7506, -37.8333, -53.15, 40.3864, -22.9661, 21.3156, 64.3833, -8.9266, 38.5167, 22.0752, -54.8, -33.5906, 32.7802, 20.7, 1.3292, -45.8667, 69.2167, 68.8193, 1.4451, 19.7297, 42.6001, -23.1203, 3.0707, 21.0964, 54.4958, -33.65, 62.456, 35.9154, -7.8447, -2.5744, -38.2667, 71.9167, -20.0064, -21.2, -2.6741, 64.8945, 54.7183, -20.3428, 43.8014, -33.9258, 36.1628, -25.2406, 49, 5.4112, 28.017, 24.6833, -42.7826, 69.4541, 71.2906, 55.7435, 70.9963, 13.2167, 37.1041, -16.4833, 70.6333, -0.6, 35.4275, 50.6789, -42.8794, 41.3712, 35.7333, -33.0153, 6.1333, 51.8, 70.0167, 57.797, 52.4443, -21.6403, 64.75, -10.1667, 4.2333, -46.2817, -38.5473, 64.7507, 45.5234, -35.7167, 69.1304, 66.8333, -33.0393, -0.8

The latitude information received is: [-46.6, 28.3667, 34.3833, -19.8, -41.1456, 36.0998, 50.0969, 68.0531, -9.8, 65.5833, -46.1927, -34.4187, 73.5069, 45.0205, 5.4802, 50.05, 71.6872, 32.6667, 20.8947, 43.7501, 46.1879, 6.7989, 56.3724, 64.5011, 16.1833, 42.097, -35.1, 24.6877, 4.375, 22.5667, 63.7506, -37.8333, -53.15, 40.3864, -22.9661, 21.3156, 64.3833, -8.9266, 38.5167, 22.0752, -54.8, -33.5906, 32.7802, 20.7, 1.3292, -45.8667, 69.2167, 68.8193, 1.4451, 19.7297, 42.6001, -23.1203, 3.0707, 21.0964, 54.4958, -33.65, 62.456, 35.9154, -7.8447, -2.5744, -38.2667, 71.9167, -20.0064, -21.2, -2.6741, 64.8945, 54.7183, -20.3428, 43.8014, -33.9258, 36.1628, -25.2406, 49, 5.4112, 28.017, 24.6833, -42.7826, 69.4541, 71.2906, 55.7435, 70.9963, 13.2167, 37.1041, -16.4833, 70.6333, -0.6, 35.4275, 50.6789, -42.8794, 41.3712, 35.7333, -33.0153, 6.1333, 51.8, 70.0167, 57.797, 52.4443, -21.6403, 64.75, -10.1667, 4.2333, -46.2817, -38.5473, 64.7507, 45.5234, -35.7167, 69.1304, 66.8333, -33.0393, -0.8

...

The latitude information received is: [-46.6, 28.3667, 34.3833, -19.8, -41.1456, 36.0998, 50.0969, 68.0531, -9.8, 65.5833, -46.1927, -34.4187, 73.5069, 45.0205, 5.4802, 50.05, 71.6872, 32.6667, 20.8947, 43.7501, 46.1879, 6.7989, 56.3724, 64.5011, 16.1833, 42.097, -35.1, 24.6877, 4.375, 22.5667, 63.7506, -37.8333, -53.15, 40.3864, -22.9661, 21.3156, 64.3833, -8.9266, 38.5167, 22.0752, -54.8, -33.5906, 32.7802, 20.7, 1.3292, -45.8667, 69.2167, 68.8193, 1.4451, 19.7297, 42.6001, -23.1203, 3.0707, 21.0964, 54.4958, -33.65, 62.456, 35.9154, -7.8447, -2.5744, -38.2667, 71.9167, -20.0064, -21.2, -2.6741, 64.8945, 54.7183, -20.3428, 43.8014, -33.9258, 36.1628, -25.2406, 49, 5.4112, 28.017, 24.6833, -42.7826, 69.4541, 71.2906, 55.7435, 70.9963, 13.2167, 37.1041, -16.4833, 70.6333, -0.6, 35.4275, 50.6789, -42.8794, 41.3712, 35.7333, -33.0153, 6.1333, 51.8, 70.0167, 57.797, 52.4443, -21.6403, 64.75, -10.1667, 4.2333, -46.2817, -38.5473, 64.7507, 45.5234, -35.7167, 69.1304, 66.8333, -33.0393, -0.8

The latitude information received is: [-46.6, 28.3667, 34.3833, -19.8, -41.1456, 36.0998, 50.0969, 68.0531, -9.8, 65.5833, -46.1927, -34.4187, 73.5069, 45.0205, 5.4802, 50.05, 71.6872, 32.6667, 20.8947, 43.7501, 46.1879, 6.7989, 56.3724, 64.5011, 16.1833, 42.097, -35.1, 24.6877, 4.375, 22.5667, 63.7506, -37.8333, -53.15, 40.3864, -22.9661, 21.3156, 64.3833, -8.9266, 38.5167, 22.0752, -54.8, -33.5906, 32.7802, 20.7, 1.3292, -45.8667, 69.2167, 68.8193, 1.4451, 19.7297, 42.6001, -23.1203, 3.0707, 21.0964, 54.4958, -33.65, 62.456, 35.9154, -7.8447, -2.5744, -38.2667, 71.9167, -20.0064, -21.2, -2.6741, 64.8945, 54.7183, -20.3428, 43.8014, -33.9258, 36.1628, -25.2406, 49, 5.4112, 28.017, 24.6833, -42.7826, 69.4541, 71.2906, 55.7435, 70.9963, 13.2167, 37.1041, -16.4833, 70.6333, -0.6, 35.4275, 50.6789, -42.8794, 41.3712, 35.7333, -33.0153, 6.1333, 51.8, 70.0167, 57.797, 52.4443, -21.6403, 64.75, -10.1667, 4.2333, -46.2817, -38.5473, 64.7507, 45.5234, -35.7167, 69.1304, 66.8333, -33.0393, -0.8

The latitude information received is: [-46.6, 28.3667, 34.3833, -19.8, -41.1456, 36.0998, 50.0969, 68.0531, -9.8, 65.5833, -46.1927, -34.4187, 73.5069, 45.0205, 5.4802, 50.05, 71.6872, 32.6667, 20.8947, 43.7501, 46.1879, 6.7989, 56.3724, 64.5011, 16.1833, 42.097, -35.1, 24.6877, 4.375, 22.5667, 63.7506, -37.8333, -53.15, 40.3864, -22.9661, 21.3156, 64.3833, -8.9266, 38.5167, 22.0752, -54.8, -33.5906, 32.7802, 20.7, 1.3292, -45.8667, 69.2167, 68.8193, 1.4451, 19.7297, 42.6001, -23.1203, 3.0707, 21.0964, 54.4958, -33.65, 62.456, 35.9154, -7.8447, -2.5744, -38.2667, 71.9167, -20.0064, -21.2, -2.6741, 64.8945, 54.7183, -20.3428, 43.8014, -33.9258, 36.1628, -25.2406, 49, 5.4112, 28.017, 24.6833, -42.7826, 69.4541, 71.2906, 55.7435, 70.9963, 13.2167, 37.1041, -16.4833, 70.6333, -0.6, 35.4275, 50.6789, -42.8794, 41.3712, 35.7333, -33.0153, 6.1333, 51.8, 70.0167, 57.797, 52.4443, -21.6403, 64.75, -10.1667, 4.2333, -46.2817, -38.5473, 64.7507, 45.5234, -35.7167, 69.1304, 66.8333, -33.0393, -0.8

The latitude information received is: [-46.6, 28.3667, 34.3833, -19.8, -41.1456, 36.0998, 50.0969, 68.0531, -9.8, 65.5833, -46.1927, -34.4187, 73.5069, 45.0205, 5.4802, 50.05, 71.6872, 32.6667, 20.8947, 43.7501, 46.1879, 6.7989, 56.3724, 64.5011, 16.1833, 42.097, -35.1, 24.6877, 4.375, 22.5667, 63.7506, -37.8333, -53.15, 40.3864, -22.9661, 21.3156, 64.3833, -8.9266, 38.5167, 22.0752, -54.8, -33.5906, 32.7802, 20.7, 1.3292, -45.8667, 69.2167, 68.8193, 1.4451, 19.7297, 42.6001, -23.1203, 3.0707, 21.0964, 54.4958, -33.65, 62.456, 35.9154, -7.8447, -2.5744, -38.2667, 71.9167, -20.0064, -21.2, -2.6741, 64.8945, 54.7183, -20.3428, 43.8014, -33.9258, 36.1628, -25.2406, 49, 5.4112, 28.017, 24.6833, -42.7826, 69.4541, 71.2906, 55.7435, 70.9963, 13.2167, 37.1041, -16.4833, 70.6333, -0.6, 35.4275, 50.6789, -42.8794, 41.3712, 35.7333, -33.0153, 6.1333, 51.8, 70.0167, 57.797, 52.4443, -21.6403, 64.75, -10.1667, 4.2333, -46.2817, -38.5473, 64.7507, 45.5234, -35.7167, 69.1304, 66.8333, -33.0393, -0.8

The latitude information received is: [-46.6, 28.3667, 34.3833, -19.8, -41.1456, 36.0998, 50.0969, 68.0531, -9.8, 65.5833, -46.1927, -34.4187, 73.5069, 45.0205, 5.4802, 50.05, 71.6872, 32.6667, 20.8947, 43.7501, 46.1879, 6.7989, 56.3724, 64.5011, 16.1833, 42.097, -35.1, 24.6877, 4.375, 22.5667, 63.7506, -37.8333, -53.15, 40.3864, -22.9661, 21.3156, 64.3833, -8.9266, 38.5167, 22.0752, -54.8, -33.5906, 32.7802, 20.7, 1.3292, -45.8667, 69.2167, 68.8193, 1.4451, 19.7297, 42.6001, -23.1203, 3.0707, 21.0964, 54.4958, -33.65, 62.456, 35.9154, -7.8447, -2.5744, -38.2667, 71.9167, -20.0064, -21.2, -2.6741, 64.8945, 54.7183, -20.3428, 43.8014, -33.9258, 36.1628, -25.2406, 49, 5.4112, 28.017, 24.6833, -42.7826, 69.4541, 71.2906, 55.7435, 70.9963, 13.2167, 37.1041, -16.4833, 70.6333, -0.6, 35.4275, 50.6789, -42.8794, 41.3712, 35.7333, -33.0153, 6.1333, 51.8, 70.0167, 57.797, 52.4443, -21.6403, 64.75, -10.1667, 4.2333, -46.2817, -38.5473, 64.7507, 45.5234, -35.7167, 69.1304, 66.8333, -33.0393, -0.8

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [11]:
# create a data frame from cities, lat, and temp
city_dict = {
    "city": cities,
    "lat": lat,
    "long": lon,
    "temp": temp,
    "temp_max": temp_max,
    "humidity": humidity,
    "cloudiness": cloudiness,
    "wind speed": wind_speed
}
city_data_df = pd.DataFrame(city_dict)
city_data_df

pd.DataFrame(dict([(k,pd.Series(v)) for k,v in city_data_df.items()])).dropna()

ValueError: arrays must all be same length

In [ ]:
# Output to csv file
city_data_df.to_csv("nithiya_output_data/cities.csv", index=False)

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [ ]:
# Create a Scatter Plot for temperature vs latitude
from datetime import date

today = date.today()
print("Today's date:", today)
x_values = city_data_df['temp']
y_values = city_data_df['lat']
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
# regress_values = x_values * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values, color="purple")
# plt.plot(x_values,regress_values,"b-")
plt.xlabel('Temperature')
plt.ylabel('Latitude')
plt.title(f'Latitude Vs Temperature Plot date: {today}', 
          fontsize=15)
plt.grid(True)
# plt.annotate(line_eq, xy=(20, 40), 
#              xycoords='data',xytext=(0.8, 0.95), 
#              textcoords='axes fraction',horizontalalignment='right', 
#              verticalalignment='top',fontsize=12,color="blue")
# print(f"The r-squared is: {rvalue**2}")

# plt.savefig("./nithiya_output_data/lat vs temp.png", bbox_inches = "tight")
plt.show()

## Latitude vs. Humidity Plot

In [ ]:
# Create a Scatter Plot for humidity vs latitude
from datetime import date

today = date.today()
print("Today's date:", today)
x_values = city_data_df['humidity']
y_values = city_data_df['lat']
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
# regress_values = x_values * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.scatter(x_values,y_values, color="purple")
# plt.plot(x_values,regress_values,"b-")
plt.scatter(x_values,y_values, color='maroon')
plt.xlabel('Humidity')
plt.ylabel('Latitude')
plt.title(f'Latitude Vs Humidity Plot date: {today}', 
          fontsize=15)
plt.grid(True)
# plt.annotate(line_eq, xy=(20, 40), 
#              xycoords='data',xytext=(0.8, 0.95), 
#              textcoords='axes fraction',horizontalalignment='right', 
#              verticalalignment='top',fontsize=12,color="blue")
# print(f"The r-squared is: {rvalue**2}")
# plt.savefig("./nithiya_output_data/lat vs humidity.png", bbox_inches = "tight")

plt.show()

## Latitude vs. Cloudiness Plot

In [ ]:
# Create a Scatter Plot for temperature vs latitude
from datetime import date

today = date.today()
print("Today's date:", today)
x_values = city_data_df['cloudiness']
y_values = city_data_df['lat']
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
# regress_values = x_values * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.scatter(x_values,y_values, color="purple")
# plt.plot(x_values,regress_values,"b-")
plt.scatter(x_values,y_values, color='grey')
plt.xlabel('Cloudiness')
plt.ylabel('Latitude')
plt.title(f'Latitude Vs Cloudiness Plot date: {today}', 
          fontsize=25)
plt.grid(True)
# plt.annotate(line_eq, xy=(20, 40), 
#              xycoords='data',xytext=(0.8, 0.95), 
#              textcoords='axes fraction',horizontalalignment='right', 
#              verticalalignment='top',fontsize=12,color="blue")
# print(f"The r-squared is: {rvalue**2}")
# plt.savefig("./nithiya_output_data/lat vs cloudiness.png", bbox_inches = "tight")

plt.show()

## Latitude vs. Wind Speed Plot

In [ ]:
# Create a Scatter Plot for temperature vs latitude
x_values = city_data_df['wind speed']
y_values = city_data_df['lat']
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
# regress_values = x_values * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.scatter(x_values,y_values, color="purple")
# plt.plot(x_values,regress_values,"b-")
plt.scatter(x_values,y_values, color='green')
plt.xlabel('Wind Speed')
plt.ylabel('Latitude')
plt.title(f'Latitude Vs Wind Speed Plot date: {today}', 
          fontsize=15)
plt.grid(True)
# plt.annotate(line_eq, xy=(20, 40), 
#              xycoords='data',xytext=(0.8, 0.95), 
#              textcoords='axes fraction',horizontalalignment='right', 
#              verticalalignment='top',fontsize=12,color="blue")
# print(f"The r-squared is: {rvalue**2}")
# plt.savefig("./nithiya_output_data/lat vs wind speed.png", bbox_inches = "tight")

plt.show()

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
north_hemisphere = city_data_df.loc[(city_data_df['lat'] >= 0)]
north_hemisphere

In [ ]:
south_hemisphere = city_data_df.loc[(city_data_df['lat'] <= 0)]
south_hemisphere

In [ ]:
# Create a Scatter Plot for max temperature vs latitude
x_values = north_hemisphere['lat']
y_values = north_hemisphere['temp_max']
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
# regress_values = x_values * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.plot(x_values,regress_values,"b-")
plt.scatter(x_values,y_values, color='green')
plt.xlabel('Latitude')
plt.ylabel('Max temp')
plt.title(f'Northern Hemisphere - Latitude Vs max temp', 
          fontsize=15)
plt.grid(True)
# plt.annotate(line_eq, xy=(20, 40), 
#              xycoords='data',xytext=(0.8, 0.95), 
#              textcoords='axes fraction',horizontalalignment='right', 
#              verticalalignment='top',fontsize=12,color="blue")
# print(f"The r-squared is: {rvalue**2}")
plt.savefig("./nithiya_output_data/north_hemi-lat vs max temp.png", bbox_inches = "tight")

plt.show()

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
# Create a Scatter Plot for max temperature vs latitude
x_values = south_hemisphere['temp_max']
y_values = south_hemisphere['lat']
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
# regress_values = x_values * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values, color="purple")
plt.plot(x_values,regress_values,"b-")
plt.xlabel('Max temp')
plt.ylabel('Latitude')
plt.title(f'Southern Hemisphere - Latitude Vs max temp', 
          fontsize=15)
plt.grid(True)
# plt.annotate(line_eq, xy=(20, 40), 
#              xycoords='data',xytext=(0.8, 0.95), 
#              textcoords='axes fraction',horizontalalignment='right', 
#              verticalalignment='top',fontsize=12,color="blue")
# print(f"The r-squared is: {rvalue**2}")
plt.savefig("./nithiya_output_data/south_hemi-lat vs max temp.png", bbox_inches = "tight")

plt.show()

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
# Create a Scatter Plot for humidity vs latitude
x_values = north_hemisphere['lat']
y_values = north_hemisphere['humidity']
plt.scatter(x_values,y_values, color='green')
plt.xlabel('Latitude')
plt.ylabel('Humidity')
plt.title(f'Northern Hemisphere - Latitude Vs Humidity', 
          fontsize=15)
plt.grid(True)
plt.savefig("./nithiya_output_data/north_hemi-lat vs humidity.png", bbox_inches = "tight")

plt.show()

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
# Create a Scatter Plot for humidity vs latitude
x_values = south_hemisphere['lat']
y_values = south_hemisphere['humidity']
plt.scatter(x_values,y_values, color='green')
plt.xlabel('Latitude')
plt.ylabel('Humidity')
plt.title(f'Southern Hemisphere - Latitude Vs Humidity', 
          fontsize=15)
plt.grid(True)
plt.savefig("./nithiya_output_data/south_hemi-lat vs humidity.png", bbox_inches = "tight")

plt.show()

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
# Create a Scatter Plot for Cloudiness vs latitude
x_values = north_hemisphere['lat']
y_values = north_hemisphere['cloudiness']
plt.scatter(x_values,y_values, color='green')
plt.xlabel('Latitude')
plt.ylabel('Cloudiness')
plt.title(f'Northern Hemisphere - Latitude Vs Cloudiness', 
          fontsize=15)
plt.grid(True)
plt.savefig("./nithiya_output_data/north_hemi-lat vs cloudiness.png", bbox_inches = "tight")

plt.show()

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
# Create a Scatter Plot for cloudiness vs latitude
x_values = south_hemisphere['lat']
y_values = south_hemisphere['humidity']
plt.scatter(x_values,y_values, color='green')
plt.xlabel('Latitude')
plt.ylabel('Cloudiness')
plt.title(f'Southern Hemisphere - Latitude Vs Cloudiness', 
          fontsize=15)
plt.grid(True)
plt.savefig("./nithiya_output_data/south_hemi-lat vs humidity.png", bbox_inches = "tight")

plt.show()

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
# Create a Scatter Plot for Wind speed vs latitude
x_values = north_hemisphere['lat']
y_values = north_hemisphere['wind_speed']
plt.scatter(x_values,y_values, color='green')
plt.xlabel('Latitude')
plt.ylabel('WindSpeed')
plt.title(f'Northern Hemisphere - Latitude Vs Wind Speed', 
          fontsize=15)
plt.grid(True)
plt.savefig("./nithiya_output_data/north_hemi-lat vs wind_speed.png", bbox_inches = "tight")

plt.show()

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
# Create a Scatter Plot for wind speed vs latitude
x_values = south_hemisphere['lat']
y_values = south_hemisphere['wind_speed']
plt.scatter(x_values,y_values, color='green')
plt.xlabel('Latitude')
plt.ylabel('Wind Speed')
plt.title(f'Northern Hemisphere - Latitude Vs Wind Speed', 
          fontsize=15)
plt.grid(True)
plt.savefig("./nithiya_output_data/south_hemi-lat vs wind_speed.png", bbox_inches = "tight")

plt.show()